In [ ]:
import numpy as np
import pandas as pd

import statistics
import random

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE

from sklearn.metrics import fbeta_score
from sklearn import metrics

import statsmodels.api as sm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

In [ ]:
q2features_trainvalid = pd.read_pickle("q2features_trainvalid.pickle")
q2target_trainvalid = pd.read_pickle("q2target_trainvalid.pickle")
q2features_test = pd.read_pickle("q2features_test.pickle")
q2target_test = pd.read_pickle("q2target_test.pickle")

Attributes with zero variance were removed from the model; these attributes did not contribute any information to the model.

In [ ]:
#Removing zero variance attributes (training set)
q2features_trainvalid = q2features_trainvalid.loc[:, q2features_trainvalid.nunique(axis=0) != 1]

In [ ]:
#Removing zero variance attributes (test set)
q2features_test = q2features_test.loc[:, q2features_test.nunique(axis=0) != 1]

Attributes with more than 10% missing data were dropped from the dataset. Dr. Iris Eekhout and colleagues in their 2013 paper titled "Missing data in a multi-item instrument were best handled by multiple imputation at the item score level" (https://doi.org/10.1016/j.jclinepi.2013.09.009) found that single-imputation methods can result in biased estimates when the dataset has a higher proportion of missing data. Single-imputation methods will be applied later in the analysis, hence the removal of these attributes with a high proportion of missing data at this step.

In [ ]:
#Removing attributes with more than 10% missing data (training set)
for col in q2features_trainvalid.columns.tolist():
  if q2features_trainvalid[col].isna().sum()/len(q2features_trainvalid[col]) > 0.10:
    q2features_trainvalid = q2features_trainvalid.drop([col], axis=1)

In [ ]:
#Removing attributes with more than 10% missing data (test set)
for col in q2features_test.columns.tolist():
  if q2features_test[col].isna().sum()/len(q2features_test[col]) > 0.10:
    q2features_test = q2features_test.drop([col], axis=1)

Imputation of missing data was performed. The alternative was to use complete case analysis, which involves removing from the dataset any rows that were missing any attributes. Complete case analysis leads to a loss of information and can lead to bias if the missing data is not missing completely at random.

Single-imputation methods were used; missing categorical values were imputed using the mode of that attribute and missing numeric values were imputed using the median. The test data set was imputed using the mode and median values of the training data set. To avoid data leakage, the training data set was not imputed using test set data.

In [ ]:
#Imputing test set
for col in q2features_test.columns.tolist():
  q2features_test[col] = q2features_test[col].replace([None], np.nan)
  if (q2features_test[col].dtype == 'category' or q2features_trainvalid[col].dtype =='datetime64[ns]'):
    q2features_test[col]= q2features_test[col].fillna(random.choice(statistics.multimode(q2features_trainvalid[col])))
  if (q2features_test[col].dtype == 'Int64' or q2features_test[col].dtype == 'int64'):
    q2features_test[col] = q2features_test[col].astype('float64')
  if (q2features_test[col].dtype == 'float64'):
    q2features_test[col] = q2features_test[col].fillna(q2features_trainvalid[col].median())

In [ ]:
#Imputing training set
for col in q2features_trainvalid.columns.tolist():
  q2features_trainvalid[col] = q2features_trainvalid[col].replace([None], np.nan)
  if (q2features_trainvalid[col].dtype == 'category' or q2features_trainvalid[col].dtype =='datetime64[ns]'):
    q2features_trainvalid[col]= q2features_trainvalid[col].fillna(random.choice(statistics.multimode(q2features_trainvalid[col])))
  if (q2features_trainvalid[col].dtype == 'Int64' or q2features_trainvalid[col].dtype == 'int64'):
    q2features_trainvalid[col] = q2features_trainvalid[col].astype('float64')
  if (q2features_trainvalid[col].dtype == 'float64'):
    q2features_trainvalid[col] = q2features_trainvalid[col].fillna(q2features_trainvalid[col].median())

Numeric attributes were standardized; variable importance will be assessed using regression models, and standardization allows for easier interpretation of the model coefficients. Standardization was performed instead of normalization, as standardization is more resistant to outliers.

In [ ]:
#Standardization  of test set
for col in q2features_test.columns.tolist():
  if (q2features_test[col].dtype == 'Int64' or q2features_test[col].dtype == 'int64' or q2features_test[col].dtype == 'float64'):
    for i in range(len(q2features_test[col])):
      q2features_test.loc[q2features_test.index[i], col] = (q2features_test.loc[q2features_test.index[i], col] - q2features_trainvalid[col].mean())/np.std(q2features_trainvalid[col])

In [ ]:
#Standardization of training set
for col in q2features_trainvalid.columns.tolist():
  if (q2features_trainvalid[col].dtype == 'Int64' or q2features_trainvalid[col].dtype == 'int64' or q2features_trainvalid[col].dtype == 'float64'):
    for i in range(len(q2features_trainvalid[col])):
      q2features_trainvalid.loc[q2features_trainvalid.index[i], col] = (q2features_trainvalid.loc[q2features_trainvalid.index[i], col] - q2features_trainvalid[col].mean())/np.std(q2features_trainvalid[col])

In [ ]:
#The SMOTE algorithm cannot handle datetime, so converting to number of
#days since January 1st, year one
for i in range(len(q2features_trainvalid['INTERVIEWDATE'])):
  q2features_trainvalid.loc[q2features_trainvalid.index[i], 'INTERVIEWDATE'] =  q2features_trainvalid.loc[q2features_trainvalid.index[i], 'INTERVIEWDATE'].toordinal()

for i in range(len(q2features_test['INTERVIEWDATE'])):
  q2features_test.loc[q2features_test.index[i], 'INTERVIEWDATE'] =  q2features_test.loc[q2features_test.index[i], 'INTERVIEWDATE'].toordinal()

q2features_trainvalid['INTERVIEWDATE'] = q2features_trainvalid['INTERVIEWDATE'].astype('float64')
q2features_test['INTERVIEWDATE'] = q2features_test['INTERVIEWDATE'].astype('float64')

In [ ]:
#Converting categorical variables to dummy variables
for col in q2features_trainvalid.columns.tolist():
  if (q2features_trainvalid[col].dtype == 'category'):
    q2features_trainvalid = pd.concat([q2features_trainvalid, pd.get_dummies(q2features_trainvalid[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q2features_trainvalid = q2features_trainvalid.drop([col], axis=1)

In [ ]:
for col in q2features_test.columns.tolist():
  if (q2features_test[col].dtype == 'category'):
    q2features_test = pd.concat([q2features_test, pd.get_dummies(q2features_test[col], prefix=col, drop_first=True, dtype='float')], axis=1)
    q2features_test = q2features_test.drop([col], axis=1)

In [ ]:
q2target_trainvalid = q2target_trainvalid.astype('bool')
q2target_trainvalid = q2target_trainvalid.astype('float')

In [ ]:
q2target_test = q2target_test.astype('bool')
q2target_test = q2target_test.astype('float')

Multicollinearity can lead to model overfitting and can artificially hide the importance of explanatory variables. Multicollinearity (and collinearity) can be assessed using variance inflation factor values. A VIF value of over 10 indicates serious multicollinearity (Vittinghoff, E., Shiboski, S., Glidden, D., & McCulloch, C. (2004). Regression Methods in Biostatistics: Linear, Logistic, Survival and Repeated Measures Models. New York:Springer. https://doi.org/10.1007/b138825). Independent variables were iteratively removed from the model starting with the variable with the highest VIF value until all VIF values were below an accceptable threshold.

In [ ]:
#Removing highly correlated variables

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('STATE')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('_AGE65YR')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('QSTVER')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('IYEAR')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('INTERVIEWDATE')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('DISPCODE')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('HTM4')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('EDUCA')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('SMOKE100')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('USENOW')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('SEQNO')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('FMONTH')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('_AGE_G')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_trainvalid.columns[q2features_trainvalid.columns.str.startswith('PRIMINSR')]
q2features_trainvalid.drop(unwanted, axis=1, inplace=True)

In [ ]:
unwanted = q2features_test.columns[q2features_test.columns.str.startswith('STATE')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('_AGE65YR')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('QSTVER')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('IYEAR')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('INTERVIEWDATE')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('DISPCODE')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('HTM4')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('EDUCA')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('SMOKE100')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('USENOW')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('SEQNO')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('FMONTH')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('_AGE_G')]
q2features_test.drop(unwanted, axis=1, inplace=True)

unwanted = q2features_test.columns[q2features_test.columns.str.startswith('PRIMINSR')]
q2features_test.drop(unwanted, axis=1, inplace=True)

Feature selection can help reduce model overfitting. Feature selection can also result in dimensionality reduction, which can improve computational efficiency. Here, a decision tree model was used to evaluate feature importance; this model has a built-in method to compute Gini importance values for each attribute. Gini importance is also referred to as "mean decrease impurity" and is a measure of how a given attribute improves the purity of a node. Attributes with a Gini importance value of less than a robust threshold of 0.01 were removed from the model. Note that dummy variables generated using one-hot encoding will be grouped together; if one variable meets the threshold then all of the set of dummy variables will remain in the model. This is because the entire set of dummy variables is required to represent one categorical variable.

In [ ]:
clf = DecisionTreeClassifier(max_depth=16, random_state=8)
clf.fit(q2features_trainvalid, q2target_trainvalid)
y_pred = clf.predict(q2features_test)

importances = clf.feature_importances_
threshold = 0.01
selected_features = q2features_trainvalid.columns[importances > threshold]
selected_features.tolist()

In [ ]:
filter_col = [col for col in q2features_trainvalid if col.startswith('') or col.startswith('')]

In [ ]:
q2features_trainvalid = q2features_trainvalid[filter_col]
q2features_test = q2features_test[filter_col]

The dataset is imbalanced; a majority of respondents reported that they did not have COVID-19 symptoms that lasted longer than 3 months. Imbalanced data adversely affects model performance; if the model encounters few instances of the minority class then it will be unable to effectively learn from this class.

The SMOTE algorithm was used to oversample the minority class; this address the issue of imbalanced data.

In [ ]:
#SMOTE Algorithm
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state = 2, k_neighbors=10)
q2features_trainvalid_SM, q2target_trainvalid_SM = smo.fit_resample(q2features_trainvalid, q2target_trainvalid.ravel())

1. Running models using hyperparameters chosen to optimize F-2 Score

Logistic Regression

In [ ]:
logReg = sm.Logit(q2target_trainvalid_SM.ravel(), q2features_trainvalid_SM,).fit()
logRegPrediction = logReg.predict(q2features_test)
logRegPrediction = np.where(logRegPrediction > 0.1, 1, 0)

f2_score = fbeta_score(q2target_test, logRegPrediction, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, logRegPrediction)
recall =  metrics.recall_score(q2target_test, logRegPrediction)
precision = metrics.precision_score(q2target_test, logRegPrediction)

In [ ]:
f2_score
#0.4854502654512672

In [ ]:
accuracy
#0.15920438732913153

In [ ]:
recall
#0.9997421351211965

In [ ]:
precision
#0.15876330876330877

In [ ]:
logRegMatrix = metrics.confusion_matrix(q2target_test, logRegPrediction)
logRegMatrix
#array([[   13, 20543],
#      [    1,  3877]])


Naive Bayes

In [ ]:
model = GaussianNB()
nbModel = model.fit(q2features_trainvalid_SM, q2target_trainvalid_SM)
nbPrediction = (nbModel.predict_proba(q2features_test)[:,0.1] >= p).astype(bool)

f2_score = fbeta_score(q2target_test, nbPrediction, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, nbPrediction)
recall =  metrics.recall_score(q2target_test, nbPrediction)
precision = metrics.precision_score(q2target_test, nbPrediction)

In [ ]:
f2_score
#0.6119546415529502

In [ ]:
accuracy
#0.6720553327330768

In [ ]:
recall
#0.8210417741103662

In [ ]:
precision
#0.3031514805293726

In [ ]:
nbMatrix = metrics.confusion_matrix(q2target_test, nbPrediction)
nbMatrix
#array([[13237,  7319],
#       [  694,  3184]])


Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=75, max_depth=10, random_state=42)
rf.fit(q2features_trainvalid_SM, q2target_trainvalid_SM)
#rfPrediction = rf.predict(q2features_trainvalid)
rfPrediction = (rf.predict_proba(q2features_test)[:,1] >= 0.2).astype(bool)

f2_score = fbeta_score(q2target_test, rfPrediction, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, rfPrediction)
recall =  metrics.recall_score(q2target_test, rfPrediction)
precision = metrics.precision_score(q2target_test, rfPrediction)

In [ ]:
f2_score
#0.5827892018321801

In [ ]:
accuracy
#0.5099860849635753

In [ ]:
recall
#0.925219185146983

In [ ]:
precision
#0.234955143736494

In [ ]:
rfMatrix = metrics.confusion_matrix(q2target_test, rfPrediction)
rfMatrix
#array([[ 8873, 11683],
#       [  290,  3588]])


Light GBM

In [ ]:
train_data = lgb.Dataset(q2features_trainvalid_SM, label=q2target_trainvalid_SM)
test_data = lgb.Dataset(q2features_test, label=q2target_test, reference=train_data)

params = {
    "num_leaves": 7,
    "reg_alpha": 0.5,
    "reg_lambda": 0.1,
    "min_data_in_leaf": 100,
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.05,
    "force_row_wise": True,
    "bagging_fraction": 0.8,
    "feature_fraction": 0.8
    }
num_round=500
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])
y_pred = bst.predict(q2features_test)
y_pred_binary = (y_pred > 0.1).astype(int)

f2_score = fbeta_score(q2target_test, y_pred_binary, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, y_pred_binary)
recall =  metrics.recall_score(q2target_test, y_pred_binary)
precision = metrics.precision_score(q2target_test, y_pred_binary)

In [ ]:
f2_score
#0.5696064026010567

In [ ]:
accuracy
#0.4652942620938037

In [ ]:
recall
#0.9396596183599794

In [ ]:
precision
#0.2211836115326252

In [ ]:
bstMatrix = metrics.confusion_matrix(q2target_test, y_pred_binary)
bstMatrix
#array([[ 7725, 12831],
#       [  234,  3644]])


2. Running models using hyperparameters chosen to optimize accuracy

Logistic Regression

In [ ]:
logReg = sm.Logit(q2target_trainvalid_SM.ravel(), q2features_trainvalid_SM,).fit()
logRegPrediction = logReg.predict(q2features_test)
logRegPrediction = np.where(logRegPrediction > 0.5, 1, 0)

f2_score = fbeta_score(q2target_test, logRegPrediction, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, logRegPrediction)
recall =  metrics.recall_score(q2target_test, logRegPrediction)
precision = metrics.precision_score(q2target_test, logRegPrediction)

In [ ]:
f2_score
#0.5856753704072819

In [ ]:
accuracy
#0.5327821887533765

In [ ]:
recall
#0.9092315626611656

In [ ]:
precision
#0.24167237834132968

In [ ]:
logRegMatrix = metrics.confusion_matrix(q2target_test, logRegPrediction)
logRegMatrix
#array([[ 9492, 11064],
#       [  352,  3526]])


Naive Bayes

In [ ]:
model = GaussianNB()
nbModel = model.fit(q2features_trainvalid_SM, q2target_trainvalid_SM)
nbPrediction = (nbModel.predict_proba(q2features_test)[:,1] >= 0.5).astype(bool)

f2_score = fbeta_score(q2target_test, nbPrediction, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, nbPrediction)
recall =  metrics.recall_score(q2target_test, nbPrediction)
precision = metrics.precision_score(q2target_test, nbPrediction)

In [ ]:
f2_score
#0.6009241665227155

In [ ]:
accuracy
#0.7562413031022346

In [ ]:
recall
#0.7176379577101599

In [ ]:
precision
#0.3640763997906855

In [ ]:
nbMatrix = metrics.confusion_matrix(q2target_test, nbPrediction)
nbMatrix
#array([[15695,  4861],
#       [ 1095,  2783]])


Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=25, max_depth=5, random_state=42)
rf.fit(q2features_trainvalid_SM, q2target_trainvalid_SM)
#rfPrediction = rf.predict(q2features_trainvalid)
rfPrediction = (rf.predict_proba(q2features_test)[:,1] >= 0.5).astype(bool)

f2_score = fbeta_score(q2target_test, rfPrediction, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, rfPrediction)
recall =  metrics.recall_score(q2target_test, rfPrediction)
precision = metrics.precision_score(q2target_test, rfPrediction)

In [ ]:
f2_score
#0.6023420674802583

In [ ]:
accuracy
#0.7837439633297864

In [ ]:
recall
#0.6923671995874162

In [ ]:
precision
#0.3962514757969303

In [ ]:
rfMatrix = metrics.confusion_matrix(q2target_test, rfPrediction)
rfMatrix
#array([[16465,  4091],
#       [ 1193,  2685]])


Light GBM

In [ ]:
train_data = lgb.Dataset(q2features_trainvalid_SM, label=q2target_trainvalid_SM)
test_data = lgb.Dataset(q2features_test, label=q2target_test, reference=train_data)

params = {
    "num_leaves": 31,
    "reg_alpha": 0.5,
    "reg_lambda": 0.5,
    "min_data_in_leaf": 100,
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.05,
    "force_row_wise": True,
    "bagging_fraction": 0.8,
    "feature_fraction": 0.8
    }
num_round=500
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])
y_pred = bst.predict(q2features_test)
y_pred_binary = (y_pred > 0.7).astype(int)

f2_score = fbeta_score(q2target_test, y_pred_binary, beta=2)
accuracy =  metrics.accuracy_score(q2target_test, y_pred_binary)
recall =  metrics.recall_score(q2target_test, y_pred_binary)
precision = metrics.precision_score(q2target_test, y_pred_binary)

In [ ]:
f2_score
#0.31934203062960864

In [ ]:
accuracy
#0.8467708930179259

In [ ]:
recall
#0.29035585353274884

In [ ]:
precision
#0.5316336166194523

In [ ]:
bstMatrix = metrics.confusion_matrix(q2target_test, y_pred_binary)
bstMatrix
#array([[19564,   992],
#       [ 2752,  1126]])


Determining Feature Importance of Optimal Model

In [ ]:
from sklearn.inspection import permutation_importance
model = GaussianNB()
nbModel = model.fit(q2features_trainvalid_SM, q2target_trainvalid_SM)
imps = permutation_importance(nbModel, q2features_test, q2target_test)
feature_names=nbModel.feature_names_in_
importances = imps.importances_mean
feature_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values('Importance', ascending=False)
print(feature_imp_df)

In [ ]:
 #      Feature      Importance
#7  ADDEPEV3_True    0.082876
#8    DECIDE_True    0.060007
#9  MEDCOST1_True    0.029197
#3       PHYSHLTH    0.000606
#4       SLEPTIM1    0.000581
#2       DROCDY4_    0.000426
#5          WTKG3    0.000344
#0       CHILDREN   -0.000082
#6         _AGE80   -0.000385
#1       CPDEMO1C   -0.000417